

# Alexa Project Code Contribution

> Author: Chet (techds on GitHub)  
> * Multinomial logistic regression:  
>   * audio data ingestion (mfcc features saved to .npy files)  
>   * model prediction  


In [1]:
# imports

from keras.utils import np_utils
from sklearn import metrics
#from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle as sh

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

# functions

def tag_cols(df,beg_str,last_col_name):
    colnames = []
    beg_str = str(beg_str)
    len_col = len(df.columns)
    for l in range(len_col-1 ):
        name = beg_str + str(l+1)
        colnames.append(name)
    last_col_name = str(last_col_name)
    colnames.append(last_col_name)
    df.columns = colnames
    
print('Ready for data ingestion and model prediction')


Using TensorFlow backend.


Ready for data ingestion and model prediction


In [4]:
# create df of training data features and labels from saved features file.
tr_mfcc = np.load('./Desktop/25_features_mfcc_train.npy')
tr_mfcc_df = pd.DataFrame(list(map(np.ravel, tr_mfcc)))
tr_mfcc_df.columns = ['feature','label']

tr_mfcc_df2 = pd.DataFrame(tr_mfcc_df['feature'].values.tolist() )
trainf_df = pd.concat([tr_mfcc_df2, tr_mfcc_df.label], axis=1)
tag_cols(trainf_df,'f','label') 

trainf_df[0:3]


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f17,f18,f19,f20,f21,f22,f23,f24,f25,label
0,-338.044529,107.832624,-8.599863,49.250348,4.054716,-2.068923,-21.844524,-16.274585,-4.601480,-2.521300,...,-8.783644,-4.230033,-3.409027,-2.882555,-2.431158,-7.108211,-0.874338,-2.797660,-7.666226,mikio_d
1,-327.008532,93.919808,-17.079697,31.987199,-6.696296,-1.553129,-11.870062,-8.161606,-6.949227,0.971683,...,-11.093544,-6.258163,-7.134278,-6.126257,-7.727422,-8.087452,-9.626957,-5.617373,-8.749785,rene_g
2,-316.934038,53.243102,-76.172196,12.368152,-3.549103,-18.639859,-44.572615,12.543678,-20.792307,-1.821228,...,-16.639423,-4.259079,-7.051992,-7.431469,-7.571922,-1.868029,-3.994799,2.889429,3.143325,rachel_m


In [5]:
# create df of training data features and labels from saved features file.
te_mfcc = np.load('./Desktop/25_features_mfcc_test.npy')
te_mfcc_df = pd.DataFrame(list(map(np.ravel, te_mfcc)))
te_mfcc_df.columns = ['feature','label']

te_mfcc_df2 = pd.DataFrame(te_mfcc_df['feature'].values.tolist() )
testf_df = pd.concat([te_mfcc_df2, te_mfcc_df.label], axis=1)
tag_cols(testf_df,'f','label')

testf_df[0:3]


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f17,f18,f19,f20,f21,f22,f23,f24,f25,label
0,-351.296923,94.099958,0.590502,34.820978,-14.364128,21.638371,-15.785959,3.455809,-14.946200,-11.285081,...,-9.326795,-7.947879,-10.988181,-9.721337,-8.737192,-3.928657,-10.108051,-2.498840,-0.955104,serhad_k
1,-293.127642,143.587180,-32.704374,42.709656,-4.666671,10.297830,-20.553964,-18.119747,-14.026293,-1.143952,...,-5.710474,-8.885007,1.352289,-5.804005,-3.664169,-1.593806,-4.117430,-1.917218,-3.832038,mikio_d
2,-392.183611,132.630387,5.576211,14.507040,4.926874,-4.470060,-11.852281,9.654318,-6.133299,2.791487,...,-2.497406,-3.244148,-0.068503,-3.993311,0.275478,-2.732089,-1.098079,0.687095,-2.713171,pooja_r


In [7]:
# prepare data
le = LabelEncoder()

x_train = np.array(tr_mfcc_df.feature.tolist())
y_train = np.array(tr_mfcc_df.label.tolist())
y_train_cat = np_utils.to_categorical(le.fit_transform(y_train))

x_test = np.array(te_mfcc_df.feature.tolist())
y_test = np.array(te_mfcc_df.label.tolist())
y_test_cat = np_utils.to_categorical(le.fit_transform(y_test))

scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print('x_train shape:', x_train.shape,'\t', 'y_train shape:', y_train.shape)
print('x_test shape :', x_test.shape,'\t', 'y_test shape :', y_test.shape)
    

x_train shape: (850, 25) 	 y_train shape: (850,)
x_test shape : (50, 25) 	 y_test shape : (50,)


In [13]:
speakers = le.classes_
num_labels = y_train_cat.shape[1]
num_features = 25


lr = LogisticRegression(solver='lbfgs', multi_class='auto')
lr.fit(x_train, y_train) 

lr_acc = lr.score(x_test, y_test) 
print('LR Test Accuracy: {:.2f}'.format(lr_acc))


LR Test Accuracy: 1.00


In [18]:
# sample voice command authentication (using cell-phone recording)

clip_path = './Desktop/flask_apps/'
filename = 'audio_m.wav'
clip = os.path.join(clip_path, filename)
print(clip)


./Desktop/flask_apps/audio_m.wav


In [21]:
from pydub import AudioSegment as audio
from pydub.playback import play 


audio_file = audio.from_wav(clip)
print('Playing ... ', clip[-12:])
play(audio_file)


Playing ...  /audio_m.wav


In [22]:
audio_dat, sample_rate = librosa.load(clip, 
                                      res_type='kaiser_fast')  

x_rt = np.mean(librosa.feature.mfcc(y = audio_dat, 
                        sr = sample_rate, 
                        n_mfcc = num_features).T,
                        axis = 0) 

x_rt_t = x_rt.reshape(1, num_features) 
x_rt_t = scaler.transform(x_rt_t)
print(x_rt_t.shape)


(1, 25)


In [23]:
rt_class = lr.predict(x_rt_t)[0] 
rt_class_prob = lr.predict_proba(x_rt_t)
rt_class_prob_rs = rt_class_prob.reshape(len(speakers),1)
rt_class_prob_df = pd.DataFrame(list(map(np.ravel, rt_class_prob_rs)))
rt_class_prob_df.columns = ['f']
rt_class_prob_df


,f
0,0.000229
1,0.005711
2,0.012177
3,0.003804
4,0.000021
5,0.619287
6,0.327575
7,0.000380
8,0.020487
9,0.010331


In [24]:
prob_thresh = .95
num_prob_thresh = 0
rt_class_prob_ls = np.array(rt_class_prob_df['f'].tolist())
max_prob = np.max(rt_class_prob_ls)
max_prob_val = round(max_prob * 100,2)
max_prob_ind = rt_class_prob_ls.tolist().index(max_prob) 
rt_pred_label = speakers[max_prob_ind]
rt_pred_label = re.sub('\_(.*)','',rt_pred_label)
print('I\'m %s%% sure that you are %s.'\
      % (max_prob_val, rt_pred_label.upper()) )


I'm 61.93% sure that you are MIKIO.
